# PyTorch와 Weights & Biases를 활용한 딥러닝 모델 모니터링

이 노트북은 PyTorch로 딥러닝 모델을 학습하면서 Weights & Biases(W&B)를 활용해 학습 과정을 모니터링하는 방법을 단계별로 보여줍니다.

## 1. 필요한 라이브러리 설치

먼저 필요한 라이브러리를 설치합니다. 실행 중 에러가 발생하면 이 셀을 실행해주세요.

In [ ]:
# 필요한 라이브러리 설치 (처음 실행할 때만 필요)
!pip install wandb tqdm matplotlib
!pip install torch torchvision

## 2. 라이브러리 임포트 및 버전 확인

필요한 라이브러리를 임포트하고 버전을 확인합니다.

In [ ]:
# 라이브러리 임포트
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, models, datasets
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import os
from tqdm import tqdm

# 버전 확인
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"사용 가능한 GPU: {torch.cuda.get_device_name(0)}")

# matplotlib 인라인 모드 설정
# %matplotlib inline # .py 파일에서는 주석 처리하거나 제거해야 함

## 3. 기본 설정

학습에 사용할 기본 설정값을 정의합니다. 필요에 따라 이 값들을 조정해보세요.

In [ ]:
# 설정값 지정 - 여기서 값을 변경하여 실험해보세요
config = {
    "learning_rate": 0.001,  # 학습률
    "batch_size": 32,      # 배치 크기
    "epochs": 5,           # 에폭 수
    "model": "ResNet18",   # 모델 아키텍처
    "optimizer": "Adam",   # 옵티마이저
    "loss": "CrossEntropyLoss",  # 손실 함수
    "num_workers": 2,      # 데이터 로딩에 사용할 스레드 수
    "device": "cuda" if torch.cuda.is_available() else "cpu"  # 학습 장치
}

print(f"학습 설정:")
for key, value in config.items():
    print(f"- {key}: {value}")

## 4. 데이터 시각화 함수

데이터를 시각적으로 확인하기 위한 함수를 정의합니다.

In [ ]:
def visualize_samples(dataloader, classes, num_samples=8):
    """
    데이터로더에서 샘플 이미지를 시각화합니다.

    Args:
        dataloader: 데이터로더
        classes: 클래스 이름 목록
        num_samples: 표시할 샘플 수
    """
    # 배치 하나를 가져옵니다
    images, labels = next(iter(dataloader))

    # 이미지와 레이블을 선택합니다
    images = images[:num_samples]
    labels = labels[:num_samples]

    # 이미지 역정규화
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])

    # 역정규화를 위한 reshape
    mean = mean.view(1, 3, 1, 1)
    std = std.view(1, 3, 1, 1)

    images_denorm = images.clone() * std + mean # 브로드캐스팅 활용

    # 이미지 시각화
    fig, axes = plt.subplots(2, num_samples//2, figsize=(15, 6))
    axes = axes.flatten()

    for i, (img, label) in enumerate(zip(images_denorm, labels)):
        # 텐서 채널 순서 변경 (C, H, W) -> (H, W, C) for matplotlib
        img_display = img.permute(1, 2, 0).numpy()
        # 값 범위 [0, 1]로 클리핑
        img_display = np.clip(img_display, 0, 1)

        axes[i].imshow(img_display)
        axes[i].set_title(f"클래스: {classes[label]}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

## 5. 데이터 준비

CIFAR-10 데이터셋을 다운로드하고 전처리합니다.

In [ ]:
# 데이터 변환 정의
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# CIFAR-10 데이터셋 로드
# 데이터셋 로드 시 에러 발생하면 download=True 유지, 이후 실행 시 False로 변경 가능
try:
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
    val_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_transform)
except Exception as e:
    print(f"데이터셋 다운로드/로드 중 오류 발생: {e}")
    print("기존 데이터가 있다면 download=False 로 시도해보세요.")
    # 필요한 경우 여기서 exit() 또는 다른 오류 처리

# 데이터 로더 생성
train_loader = DataLoader(
    train_dataset,
    batch_size=config["batch_size"],
    shuffle=True,
    num_workers=config["num_workers"],
    pin_memory=True if config["device"] == "cuda" else False # GPU 사용 시 pin_memory=True 권장
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config["batch_size"],
    shuffle=False,
    num_workers=config["num_workers"],
    pin_memory=True if config["device"] == "cuda" else False
)

# 클래스 이름
classes = ['비행기', '자동차', '새', '고양이', '사슴', '개', '개구리', '말', '배', '트럭']

print(f"학습 데이터셋: {len(train_dataset)} 이미지")
print(f"검증 데이터셋: {len(val_dataset)} 이미지")
print(f"클래스: {classes}")

## 6. 데이터 샘플 시각화

학습 데이터의 일부를 시각화하여 확인해봅니다.

In [ ]:
# 학습 데이터 샘플 시각화 (데이터 로더가 정상 생성되었을 때만 실행)
if 'train_loader' in locals():
     visualize_samples(train_loader, classes)
else:
     print("학습 데이터 로더가 준비되지 않아 시각화를 건너<0xEB>고 뜁니다.")

## 7. WandbMonitor 클래스 정의

Weights & Biases를 활용한 모니터링을 위한 클래스를 정의합니다.

In [ ]:
class WandbMonitor:
    def __init__(self, project_name, config):
        """
        W&B 모니터링 초기화

        Args:
            project_name (str): W&B 프로젝트 이름
            config (dict): 설정 딕셔너리
        """
        try:
            # W&B 초기화
            self.run = wandb.init(
                project=project_name,
                config=config,
                name=f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                reinit=True # 혹시 모를 중복 초기화 방지
            )
            print(f"W&B Run 시작: {self.run.name} (Project: {project_name})")
            # 설정 저장
            self.config = wandb.config
        except Exception as e:
            print(f"W&B 초기화 중 오류 발생: {e}")
            print("W&B 로그인이 되어있는지, API 키가 유효한지 확인하세요.")
            self.run = None # 초기화 실패 시 run 객체 None 설정
            self.config = config # 기본 config 유지

    def watch_model(self, model, log_freq=100):
        """ 그라디언트 로깅을 위한 모델 감시 """
        if self.run: # run이 성공적으로 초기화되었을 때만 실행
            try:
                wandb.watch(model, log="all", log_freq=log_freq) # log="all" 로 가중치/그라디언트 모두 로깅
            except Exception as e:
                 print(f"W&B 모델 감시 중 오류: {e}")


    def log_metrics(self, metrics, step=None):
        """ 메트릭을 W&B에 로깅 """
        if self.run:
            try:
                wandb.log(metrics, step=step)
            except Exception as e:
                 print(f"W&B 메트릭 로깅 중 오류: {e}")


    def log_images(self, images, name="images", captions=None):
       """ 이미지를 W&B에 로깅 """
       if self.run:
            try:
                # 이미지 텐서 처리 (CPU 이동, numpy 변환, 정규화 복원 대신 [0,1] 클램핑)
                images_to_log = images.cpu().clone() # 원본 변경 방지 위해 clone
                # 역정규화 (시각화 함수와 유사하게)
                mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
                std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
                images_to_log = images_to_log * std + mean
                images_to_log = torch.clamp(images_to_log, 0, 1) # [0, 1] 범위로 클램핑

                wandb_images = []
                for i, img_tensor in enumerate(images_to_log):
                    caption = captions[i] if captions and i < len(captions) else None
                    wandb_images.append(wandb.Image(img_tensor, caption=caption))

                wandb.log({name: wandb_images})
            except Exception as e:
                print(f"W&B 이미지 로깅 중 오류: {e}")


    def log_confusion_matrix(self, y_true, y_pred, class_names):
        """ 혼동 행렬을 W&B에 로깅 """
        if self.run:
             try:
                wandb.log({
                    "confusion_matrix": wandb.plot.confusion_matrix(
                        y_true=y_true,
                        preds=y_pred,
                        class_names=class_names
                    )
                })
             except Exception as e:
                 print(f"W&B 혼동 행렬 로깅 중 오류: {e}")


    def log_learning_rate(self, optimizer, step):
        """ 학습률을 W&B에 로깅 """
        if self.run:
            try:
                 # 옵티마이저 종류에 따라 학습률 접근 방식이 다를 수 있음 (일반적인 경우)
                if optimizer and optimizer.param_groups:
                     lr = optimizer.param_groups[0]['lr']
                     wandb.log({"learning_rate": lr}, step=step)
                else:
                     print("경고: 유효한 옵티마이저가 아니거나 param_groups가 없어 학습률 로깅 불가")
            except Exception as e:
                print(f"W&B 학습률 로깅 중 오류: {e}")


    # log_gradients 와 log_model_weights는 wandb.watch(log="all") 사용 시 자동 로깅될 수 있음
    # 수동 로깅이 필요하다면 아래 함수 사용
    def log_gradients_manual(self, model, step):
        """ 그라디언트 통계를 수동으로 W&B에 로깅 """
        if self.run:
            grads = {f"gradients/{name}": wandb.Histogram(param.grad.cpu())
                     for name, param in model.named_parameters()
                     if param.requires_grad and param.grad is not None}
            if grads:
                try:
                    wandb.log(grads, step=step)
                except Exception as e:
                    print(f"W&B 그라디언트 로깅 중 오류: {e}")

    def log_weights_manual(self, model, step):
        """ 모델 가중치 통계를 수동으로 W&B에 로깅 """
        if self.run:
            weights = {f"weights/{name}": wandb.Histogram(param.data.cpu())
                       for name, param in model.named_parameters()
                       if param.requires_grad}
            if weights:
                try:
                    wandb.log(weights, step=step)
                except Exception as e:
                    print(f"W&B 가중치 로깅 중 오류: {e}")


    def log_artifact(self, name, type, description, path):
        """ 아티팩트를 W&B에 로깅 """
        if self.run and os.path.exists(path): # 파일 존재 확인 추가
            try:
                artifact = wandb.Artifact(name, type=type, description=description)
                artifact.add_file(path)
                wandb.log_artifact(artifact)
            except Exception as e:
                print(f"W&B 아티팩트 로깅 중 오류: {e}")
        elif not os.path.exists(path):
             print(f"경고: 아티팩트 파일 '{path}'를 찾을 수 없어 로깅을 건너<0xEB>고 뜁니다.")


    def finish(self):
        """ W&B 실행 종료 """
        if self.run:
            try:
                wandb.finish()
                print("W&B Run 종료.")
                self.run = None # 종료 후 run 객체 None 설정
            except Exception as e:
                print(f"W&B 종료 중 오류 발생: {e}")

## 8. 모델 초기화 및 확인

사전 학습된 ResNet18 모델을 로드하고 마지막 레이어를 수정합니다.

In [ ]:
# 모델 초기화
# pretrained=True 대신 weights 사용 권장 (최신 torchvision)
try:
    weights = models.ResNet18_Weights.DEFAULT # DEFAULT는 ImageNet 1K v1 가중치
    model = models.resnet18(weights=weights)
    train_transform = weights.transforms() # 사전 학습된 가중치에 맞는 변환 사용 권장
    val_transform = weights.transforms()
    print("최신 방식으로 ResNet18 가중치 로드 및 변환 사용.")
except AttributeError:
    print("경고: 최신 ResNet18_Weights 방식 사용 불가. pretrained=True 사용.")
    model = models.resnet18(pretrained=True)
    # 이전 변환 방식 유지 (위 셀에서 정의된 것 사용)

num_ftrs = model.fc.in_features
# 마지막 레이어(Fully Connected)를 CIFAR-10 클래스 수(10)에 맞게 교체
model.fc = nn.Linear(num_ftrs, len(classes))

# --- 전이 학습 전략 ---
# 옵션 1: 마지막 레이어만 학습 (Feature Extractor) - 기본
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters(): # 새로 추가된 FC 레이어만 학습
    param.requires_grad = True
print("전이 학습 전략: 마지막 레이어만 학습 (Feature Extractor)")

# 옵션 2: 전체 모델 미세 조정 (Fine-tuning) - 필요시 주석 해제
# print("전이 학습 전략: 전체 모델 미세 조정 (Fine-tuning)")
# for param in model.parameters():
#     param.requires_grad = True

# 모델을 지정된 장치로 이동
device = torch.device(config["device"])
model = model.to(device)

# 학습 가능한 파라미터 수 계산 함수
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"모델 아키텍처: {config['model']}")
print(f"학습 가능한 파라미터 수: {count_parameters(model):,}")
# print(f"모델 전체 파라미터 수: {sum(p.numel() for p in model.parameters()):,}") # 전체 파라미터
print(f"마지막 FC 레이어: Input={num_ftrs}, Output={len(classes)}")


# 모델 구조 시각화 (선택 사항)
# 모델이 복잡하면 시각화가 어려울 수 있음
print("모델 구조:")
print(model)

## 9. 학습 함수 정의

모델을 학습시키는 함수를 정의합니다. 이 함수는 W&B 모니터링을 활용하여 학습 과정을 추적합니다.

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, monitor, num_epochs, device, classes):
    """
    W&B 모니터링을 사용하여 모델 학습 및 검증

    Args:
        (이전과 동일)

    Returns:
        history (dict): 에폭별 학습/검증 손실 및 정확도 기록
        best_model_path (str): 최고 검증 정확도를 달성한 모델의 저장 경로
    """
    best_val_acc = 0.0
    best_model_path = 'best_model_checkpoint.pth' # 체크포인트 파일명
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    # W&B가 초기화되었는지 확인
    if not monitor or not monitor.run:
         print("경고: W&B 모니터가 초기화되지 않아 로깅 없이 학습을 진행합니다.")
         use_wandb = False
    else:
         use_wandb = True

    total_steps = 0 # 전체 배치 스텝 카운트

    for epoch in range(num_epochs):
        # --- 학습 단계 ---
        model.train() # 모델을 학습 모드로 설정
        running_loss = 0.0
        running_corrects = 0
        processed_samples = 0

        # tqdm으로 진행률 표시
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Train]', leave=False)

        for inputs, labels in train_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            # 옵티마이저 그래디언트 초기화
            optimizer.zero_grad()

            # 순전파
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # 예측 (가장 높은 확률을 가진 클래스 인덱스)
            _, preds = torch.max(outputs, 1)

            # 역전파 및 옵티마이저 스텝
            loss.backward()
            optimizer.step()

            # 통계 업데이트
            batch_loss = loss.item() * inputs.size(0) # 배치 손실 (평균 * 배치크기)
            batch_corrects = torch.sum(preds == labels.data)
            running_loss += batch_loss
            running_corrects += batch_corrects
            processed_samples += inputs.size(0)
            total_steps += 1

            # tqdm 진행률 표시줄 업데이트 (배치 평균 손실/정확도)
            train_bar.set_postfix(loss=f"{batch_loss/inputs.size(0):.4f}", acc=f"{batch_corrects.double()/inputs.size(0)*100:.2f}%")

            # 배치 단위 메트릭 W&B 로깅 (선택 사항, 너무 자주하면 느려짐)
            if use_wandb and total_steps % 100 == 0: # 예: 100 스텝마다 로깅
                 monitor.log_metrics({
                     'batch_train_loss': batch_loss / inputs.size(0),
                     'batch_train_acc': batch_corrects.double() / inputs.size(0) * 100
                 }, step=total_steps)


        # 에폭 학습 통계 계산
        epoch_train_loss = running_loss / processed_samples
        epoch_train_acc = running_corrects.double() / processed_samples * 100
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)

        # --- 검증 단계 ---
        model.eval() # 모델을 평가 모드로 설정
        running_loss = 0.0
        running_corrects = 0
        processed_samples = 0
        all_preds = []
        all_labels = []

        val_bar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} [Val]', leave=False)

        with torch.no_grad(): # 검증 시에는 그래디언트 계산 비활성화
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                processed_samples += inputs.size(0)

                # 혼동 행렬 계산을 위해 예측과 실제 레이블 저장
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                # tqdm 진행률 표시줄 업데이트 (배치 평균 손실/정확도)
                val_bar.set_postfix(loss=f"{loss.item():.4f}", acc=f"{torch.sum(preds == labels.data).double()/inputs.size(0)*100:.2f}%")

        # 에폭 검증 통계 계산
        epoch_val_loss = running_loss / processed_samples
        epoch_val_acc = running_corrects.double() / processed_samples * 100
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)

        print(f"
Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {epoch_train_loss:.4f} Acc: {epoch_train_acc:.2f}%")
        print(f"Val Loss:   {epoch_val_loss:.4f} Acc: {epoch_val_acc:.2f}%")

        # --- W&B 로깅 (에폭 단위) ---
        if use_wandb:
            log_data = {
                'epoch': epoch + 1,
                'train_loss': epoch_train_loss,
                'train_acc': epoch_train_acc,
                'val_loss': epoch_val_loss,
                'val_acc': epoch_val_acc
            }
            monitor.log_metrics(log_data, step=epoch) # step을 epoch 기준으로 로깅
            monitor.log_learning_rate(optimizer, epoch) # 학습률 로깅

            # 혼동 행렬 로깅 (예: 마지막 에폭 또는 주기적으로)
            if epoch == num_epochs - 1: # 마지막 에폭에만 로깅
                 monitor.log_confusion_matrix(np.array(all_labels), np.array(all_preds), classes)

            # 예시 이미지 로깅 (예: 마지막 에폭 또는 주기적으로)
            if epoch == num_epochs - 1:
                 # 검증 데이터의 마지막 배치 사용
                 monitor.log_images(inputs[:8], name="validation_samples", captions=[f"Pred:{classes[p]}, True:{classes[l]}" for p, l in zip(all_preds[-8:], all_labels[-8:])])


        # 최고 검증 정확도 모델 저장
        if epoch_val_acc > best_val_acc:
            print(f"Validation accuracy improved ({best_val_acc:.2f}% -> {epoch_val_acc:.2f}%). Saving model...")
            best_val_acc = epoch_val_acc
            # 모델 상태 저장 (state_dict)
            torch.save(model.state_dict(), best_model_path)
            print(f"Model saved to {best_model_path}")
            # W&B 아티팩트로 모델 저장 (선택 사항)
            if use_wandb:
                 monitor.log_artifact(
                     'best_model', # 아티팩트 논리적 이름
                     'model',      # 아티팩트 타입
                     f'Epoch {epoch+1} best model with val_acc {best_val_acc:.2f}%',
                     best_model_path
                 )
        print("-" * 30)


    print(f"
학습 완료. 최고 검증 정확도: {best_val_acc:.2f}%")
    return history, best_model_path

## 10. 옵티마이저 및 손실 함수 초기화

모델 학습에 필요한 옵티마이저와 손실 함수를 초기화합니다.

In [ ]:
# 옵티마이저 선택 및 초기화
if config["optimizer"].lower() == "adam":
    # 마지막 레이어의 파라미터만 학습 대상으로 전달
    optimizer = optim.Adam(model.fc.parameters(), lr=config["learning_rate"])
elif config["optimizer"].lower() == "sgd":
    optimizer = optim.SGD(model.fc.parameters(), lr=config["learning_rate"], momentum=0.9)
else:
    print(f"경고: 지원되지 않는 옵티마이저 '{config['optimizer']}'. Adam을 기본값으로 사용합니다.")
    optimizer = optim.Adam(model.fc.parameters(), lr=config["learning_rate"])


# 손실 함수 초기화
if config["loss"].lower() == "crossentropy":
    criterion = nn.CrossEntropyLoss()
else:
     print(f"경고: 지원되지 않는 손실 함수 '{config['loss']}'. CrossEntropyLoss를 기본값으로 사용합니다.")
     criterion = nn.CrossEntropyLoss()


print(f"옵티마이저: {type(optimizer).__name__} (학습률: {config['learning_rate']})")
print(f"손실 함수: {type(criterion).__name__}")

## 11. Weights & Biases 설정

Weights & Biases 모니터링을 설정합니다. 이 부분을 실행하면 W&B에 로그인해야 할 수 있습니다.

In [ ]:
# W&B 모니터 초기화
# wandb.login() # 주피터 노트북 환경에서 명시적으로 로그인 필요 시 주석 해제
monitor = WandbMonitor("pytorch-cifar10-wandb", config) # 프로젝트 이름 지정

# 모델 파라미터 및 그라디언트 모니터링 시작 (WandbMonitor 내부에서 처리)
if monitor.run: # monitor가 성공적으로 초기화되었을 때만 watch 실행
    monitor.watch_model(model, log_freq=100) # 100 스텝마다 로깅

    print("Weights & Biases 모니터링 설정 완료.")
    print(f"Run Name: {monitor.run.name}")
    print(f"W&B 대시보드에서 '{monitor.run.project}/{monitor.run.id}' 를 확인하세요.")
else:
    print("W&B 모니터링 설정 실패. 로깅 없이 진행됩니다.")

## 12. 모델 학습 시작

모델 학습을 시작합니다. 이 과정은 설정에 따라 시간이 소요될 수 있습니다.

In [ ]:
# 모델 학습 실행
# 필요한 모든 변수(model, loaders, criterion, optimizer 등)가 정의되어 있어야 함
try:
    history, best_model_path = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        monitor=monitor, # WandbMonitor 인스턴스 전달
        num_epochs=config["epochs"],
        device=device,
        classes=classes
    )
    print(f"
학습 완료! 최고 성능 모델이 '{best_model_path}'에 저장되었습니다.")

except NameError as e:
    print(f"오류: 학습 시작 전 필요한 변수가 정의되지 않았습니다. ({e})")
    print("이전 셀들이 모두 성공적으로 실행되었는지 확인하세요.")
except Exception as e:
    print(f"학습 중 예상치 못한 오류 발생: {e}")
    # 오류 발생 시에도 W&B run 종료 시도
    if 'monitor' in locals() and monitor.run:
        monitor.finish()

## 13. 학습 결과 분석

학습이 완료된 후 성능을 분석합니다.

In [ ]:
# 학습 결과 시각화 및 분석 (history 변수가 정상적으로 반환되었을 경우)
if 'history' in locals() and history['val_acc']: # history가 있고 검증 정확도 기록이 있을 때
    best_epoch_idx = np.argmax(history['val_acc']) # 최고 정확도의 인덱스 (0부터 시작)
    best_val_acc = history['val_acc'][best_epoch_idx]
    best_epoch_num = best_epoch_idx + 1 # 에폭 번호 (1부터 시작)

    epochs_range = range(1, len(history['train_loss']) + 1)

    plt.figure(figsize=(14, 6))

    # 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, history['train_loss'], 'bo-', label='학습 손실')
    plt.plot(epochs_range, history['val_loss'], 'ro-', label='검증 손실')
    plt.axvline(best_epoch_num, color='g', linestyle='--', label=f'최고 성능 에폭 ({best_epoch_num})')
    plt.title('학습 및 검증 손실')
    plt.xlabel('에폭')
    plt.ylabel('손실')
    plt.legend()
    plt.grid(True)

    # 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, history['train_acc'], 'bo-', label='학습 정확도')
    plt.plot(epochs_range, history['val_acc'], 'ro-', label='검증 정확도')
    plt.axvline(best_epoch_num, color='g', linestyle='--', label=f'최고 성능 에폭 ({best_epoch_num})')
    plt.title('학습 및 검증 정확도')
    plt.xlabel('에폭')
    plt.ylabel('정확도 (%)')
    plt.legend()
    plt.grid(True)

    plt.suptitle('학습 결과 분석', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # suptitle과의 간격 조정
    plt.show()

    # 최고 성능 출력
    print(f"
최고 성능 (에폭 {best_epoch_num}):")
    print(f"  - 검증 정확도: {best_val_acc:.2f}%")
    print(f"  - 검증 손실: {history['val_loss'][best_epoch_idx]:.4f}")
    print(f"  - 학습 정확도: {history['train_acc'][best_epoch_idx]:.2f}%")
    print(f"  - 학습 손실: {history['train_loss'][best_epoch_idx]:.4f}")
else:
    print("학습 기록(history)이 없거나 유효하지 않아 결과를 분석할 수 없습니다.")

## 14. 최고 성능 모델 평가

저장된 최고 성능 모델을 로드하고 검증 데이터셋에 대해 최종 평가를 수행합니다.

In [ ]:
# 최고 성능 모델 로드 및 평가
if 'best_model_path' in locals() and os.path.exists(best_model_path):
    print(f"최고 성능 모델 '{best_model_path}' 로드 중...")
    # 모델 구조 다시 정의 (저장된 state_dict와 구조가 일치해야 함)
    eval_model = models.resnet18(weights=None) # 가중치 없이 구조만 가져옴
    num_ftrs = eval_model.fc.in_features
    eval_model.fc = nn.Linear(num_ftrs, len(classes))

    # state_dict 로드
    eval_model.load_state_dict(torch.load(best_model_path, map_location=device)) # map_location으로 장치 지정
    eval_model = eval_model.to(device)
    eval_model.eval() # 평가 모드 설정
    print("모델 로드 완료.")

    # 검증 데이터셋에서 성능 평가
    final_correct = 0
    final_total = 0
    final_predictions = []
    final_true_labels = []

    print("최종 모델 평가 시작...")
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="최종 모델 평가"):
            images, labels = images.to(device), labels.to(device)
            outputs = eval_model(images)
            _, predicted = torch.max(outputs.data, 1)
            final_total += labels.size(0)
            final_correct += (predicted == labels).sum().item()

            # 예측 및 실제 레이블 저장 (혼동 행렬, 클래스별 정확도용)
            final_predictions.extend(predicted.cpu().numpy())
            final_true_labels.extend(labels.cpu().numpy())

    # 최종 정확도 계산
    final_accuracy = 100 * final_correct / final_total
    print(f'
최고 성능 모델의 최종 검증 정확도: {final_accuracy:.2f}%')

    # 클래스별 정확도 계산
    class_correct = np.zeros(len(classes))
    class_total = np.zeros(len(classes))
    for i in range(len(final_true_labels)):
        label = final_true_labels[i]
        pred = final_predictions[i]
        if label == pred:
            class_correct[label] += 1
        class_total[label] += 1

    # 클래스별 정확도 시각화
    plt.figure(figsize=(12, 6))
    # 0으로 나누는 경우 방지 (해당 클래스 샘플이 없을 경우)
    class_accuracies = [100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
                        for i in range(len(classes))]
    plt.bar(range(len(classes)), class_accuracies, align='center', alpha=0.7, color='skyblue')
    plt.xticks(range(len(classes)), classes, rotation=45, ha='right')
    plt.ylabel('정확도 (%)')
    plt.title('최고 성능 모델의 클래스별 정확도')
    plt.grid(True, axis='y', linestyle='--', alpha=0.6)

    # 막대 위에 정확도 값 표시
    for i, v in enumerate(class_accuracies):
        plt.text(i, v + 1, f'{v:.1f}%', ha='center', va='bottom', fontsize=9)

    plt.ylim(0, 105) # y축 범위 살짝 늘리기
    plt.tight_layout()
    plt.show()

    # W&B에 최종 결과 로깅 (선택 사항)
    if 'monitor' in locals() and monitor.run:
         monitor.log_metrics({
             "final_val_accuracy": final_accuracy,
             "class_accuracies": wandb.Table(data=[[cls, acc] for cls, acc in zip(classes, class_accuracies)],
                                              columns=["Class", "Accuracy"])
         })
         # 최종 혼동 행렬도 로깅 가능
         monitor.log_confusion_matrix(np.array(final_true_labels), np.array(final_predictions), classes)

else:
    print("최고 성능 모델 경로(best_model_path)를 찾을 수 없거나 파일이 존재하지 않습니다.")
    print("모델 평가를 건너<0xEB>고 뜁니다.")

## 15. 예측 시각화

검증 데이터셋에서 몇 가지 예시 이미지에 대한 최고 성능 모델의 예측을 시각화합니다.

In [ ]:
# 예측 시각화 함수 정의 (이전 셀에서 정의됨, 여기서는 실행만)
if 'eval_model' in locals(): # 평가 모델이 로드되었는지 확인
    print("최고 성능 모델을 사용한 예측 시각화:")
    # visualize_predictions 함수는 이전에 정의되었다고 가정
    # 이 함수 내부에서 모델, 로더, 클래스, 샘플 수를 인자로 받음
    # visualize_samples 함수와 유사하지만 예측 결과도 함께 표시
    def visualize_model_predictions(model, dataloader, classes, device, num_samples=8):
        model.eval()
        images, labels = next(iter(dataloader))
        images, labels = images[:num_samples], labels[:num_samples]
        images_gpu = images.to(device)

        with torch.no_grad():
            outputs = model(images_gpu)
            _, preds = torch.max(outputs, 1)
            probs = torch.nn.functional.softmax(outputs, dim=1)

        # 역정규화
        mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)
        images_denorm = images * std + mean
        images_denorm = torch.clamp(images_denorm, 0, 1)

        fig, axes = plt.subplots(2, num_samples // 2, figsize=(16, 8))
        axes = axes.flatten()

        for i, (img, label, pred, prob) in enumerate(zip(images_denorm, labels, preds.cpu(), probs.cpu())):
            img_display = img.permute(1, 2, 0).numpy()
            axes[i].imshow(img_display)
            is_correct = label == pred
            title_color = 'green' if is_correct else 'red'
            pred_prob = prob[pred].item() # 예측된 클래스의 확률
            axes[i].set_title(f"실제: {classes[label]}
예측: {classes[pred]} ({pred_prob:.2f})",
                              color=title_color, fontsize=10)
            axes[i].axis('off')

        plt.tight_layout()
        plt.show()

    # 함수 실행
    visualize_model_predictions(eval_model, val_loader, classes, device)

else:
    print("평가 모델(eval_model)이 로드되지 않아 예측 시각화를 건너<0xEB>고 뜁니다.")

## 16. 혼동 행렬 시각화

최고 성능 모델의 예측 결과에 대한 혼동 행렬을 시각화합니다.

In [ ]:
# 혼동 행렬 시각화 (sklearn, seaborn 사용)
if 'final_true_labels' in locals() and 'final_predictions' in locals():
    try:
        from sklearn.metrics import confusion_matrix
        import seaborn as sns

        conf_mat = confusion_matrix(final_true_labels, final_predictions)

        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
                    xticklabels=classes, yticklabels=classes, annot_kws={"size": 10})
        plt.xlabel('예측된 레이블', fontsize=12)
        plt.ylabel('실제 레이블', fontsize=12)
        plt.title('혼동 행렬 (Confusion Matrix)', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

        # 정규화된 혼동 행렬 (클래스별 비율)
        conf_mat_norm = conf_mat.astype('float') / conf_mat.sum(axis=1)[:, np.newaxis]
        conf_mat_norm = np.nan_to_num(conf_mat_norm) # NaN 값을 0으로 처리

        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_mat_norm, annot=True, fmt='.2f', cmap='viridis', # 다른 컬러맵 사용 예시
                    xticklabels=classes, yticklabels=classes, annot_kws={"size": 10})
        plt.xlabel('예측된 레이블', fontsize=12)
        plt.ylabel('실제 레이블', fontsize=12)
        plt.title('정규화된 혼동 행렬 (Normalized Confusion Matrix)', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()

    except ImportError:
        print("오류: 혼동 행렬 시각화를 위해 scikit-learn과 seaborn 라이브러리가 필요합니다.")
        print("!pip install scikit-learn seaborn")
    except Exception as e:
        print(f"혼동 행렬 시각화 중 오류 발생: {e}")
else:
    print("혼동 행렬을 계산하기 위한 예측 결과(final_predictions) 또는 실제 레이블(final_true_labels)이 없습니다.")

## 17. 모델 해석 (Grad-CAM)

Grad-CAM 기법을 사용하여 모델이 이미지의 어떤 영역에 주목하여 예측하는지 시각화합니다.

In [ ]:
# Grad-CAM 시각화
# 필요한 라이브러리 설치 확인
try:
    from pytorch_grad_cam import GradCAM
    from pytorch_grad_cam.utils.image import show_cam_on_image
    from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
    grad_cam_available = True
except ImportError:
    print("경고: Grad-CAM 시각화를 위해 'grad-cam' 라이브러리가 필요합니다.")
    print("!pip install grad-cam")
    grad_cam_available = False

if grad_cam_available and 'eval_model' in locals():
    print("Grad-CAM 시각화 생성 중...")
    # Grad-CAM 대상 레이어 선택 (모델 구조에 따라 변경 필요)
    # 예: ResNet18의 마지막 conv 블록의 마지막 레이어
    try:
        target_layers = [eval_model.layer4[-1]] # ResNet 구조에 따라 변경
    except AttributeError:
         print("오류: 모델 구조에서 'layer4'를 찾을 수 없습니다. 대상 레이어를 확인하세요.")
         target_layers = None # 대상 레이어 설정 실패

    if target_layers:
        # 이미지 가져오기 (검증 데이터 로더 사용, num_samples=4)
        dataiter = iter(val_loader)
        images, labels = next(dataiter)
        images_selected = images[:4].to(device) # GPU로 이동
        labels_selected = labels[:4]

        # 원본 이미지 준비 (역정규화)
        mean = torch.tensor([0.485, 0.456, 0.406], device=device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=device).view(1, 3, 1, 1)
        images_denorm = images_selected.clone() * std + mean
        images_denorm = torch.clamp(images_denorm, 0, 1)

        # Grad-CAM 객체 초기화
        cam = GradCAM(model=eval_model, target_layers=target_layers, use_cuda=torch.cuda.is_available())

        fig, axes = plt.subplots(2, 4, figsize=(16, 8)) # 가로 크기 늘림

        for i, (img_tensor, label) in enumerate(zip(images_selected, labels_selected)):
            # 원본 이미지 표시 (CPU로 이동 및 채널 변경)
            img_np = images_denorm[i].cpu().permute(1, 2, 0).numpy()
            axes[0, i].imshow(img_np)
            axes[0, i].set_title(f'원본: {classes[label]}')
            axes[0, i].axis('off')

            # Grad-CAM 계산 (개별 이미지에 대해)
            input_tensor = img_tensor.unsqueeze(0) # 배치 차원 추가
            # 타겟 설정: 분류 모델의 경우 예측된 클래스를 타겟으로 하거나 실제 레이블을 타겟으로 할 수 있음
            # targets = [ClassifierOutputTarget(label.item())] # 실제 레이블 기준
            # 또는 모델 예측 기준:
            with torch.no_grad():
                 output = eval_model(input_tensor)
                 _, pred_idx = torch.max(output, 1)
            targets = [ClassifierOutputTarget(pred_idx.item())] # 예측 레이블 기준


            grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
            grayscale_cam = grayscale_cam[0, :] # 배치 차원 제거, 결과는 (H, W) 형태

            # Grad-CAM 오버레이 및 표시
            try:
                cam_image = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)
                axes[1, i].imshow(cam_image)
                axes[1, i].set_title(f'Grad-CAM (예측: {classes[pred_idx.item()]})')
                axes[1, i].axis('off')
            except Exception as vis_e:
                 print(f"Grad-CAM 시각화 중 오류 (이미지 {i}): {vis_e}")
                 axes[1, i].set_title('Grad-CAM 시각화 오류')
                 axes[1, i].axis('off')


        plt.tight_layout()
        plt.suptitle('Grad-CAM: 모델이 주목하는 영역 시각화', fontsize=16, y=1.03)
        plt.show()

elif not grad_cam_available:
    print("Grad-CAM 라이브러리가 없어 시각화를 건너<0xEB>고 뜁니다.")
else: # 'eval_model'이 없는 경우
    print("평가 모델(eval_model)이 로드되지 않아 Grad-CAM 시각화를 건너<0xEB>고 뜁니다.")

## 18. 모니터링 종료

Weights & Biases 모니터링 실행을 종료합니다.

In [ ]:
# W&B 모니터링 종료
if 'monitor' in locals() and monitor.run: # monitor 객체가 있고, run이 활성 상태일 때만 종료 시도
    monitor.finish()
else:
    print("W&B 모니터가 초기화되지 않았거나 이미 종료된 상태입니다.")

## 19. 커스텀 이미지 예측

로컬 또는 업로드된 이미지에 대해 학습된 모델로 예측을 수행합니다.

In [ ]:
# 커스텀 이미지 예측 함수 정의 (이전 셀에서 정의됨, 여기서는 실행만)

# --- 사용자 입력 또는 파일 업로드를 통해 이미지 경로 설정 ---
# 예시 1: 직접 경로 입력
# custom_image_path = "your_image.jpg" # 실제 이미지 경로로 변경하세요.

# 예시 2: Colab 환경에서 파일 업로드
# try:
#     from google.colab import files
#     uploaded = files.upload()
#     if uploaded:
#         custom_image_path = list(uploaded.keys())[0]
#         print(f"업로드된 파일: {custom_image_path}")
#     else:
#         custom_image_path = None
#         print("파일이 업로드되지 않았습니다.")
# except ImportError:
#      custom_image_path = None
#      print("Colab 환경이 아니므로 파일 업로드를 건너<0xEB>고 뜁니다. 직접 경로를 지정하세요.")

custom_image_path = None # 기본적으로 경로 없음으로 시작

# --- 예측 실행 ---
if custom_image_path and 'eval_model' in locals() and os.path.exists(custom_image_path):
    print(f"
'{custom_image_path}' 이미지 예측 수행:")
    # predict_image 함수는 이전에 정의되었다고 가정
    # 필요한 인자: 모델, 이미지 경로, 클래스 리스트, 장치
    def predict_custom_image(model, image_path, classes, device, transform=None):
        """ 커스텀 이미지 예측 및 시각화 함수 (재정의 또는 이전 함수 사용) """
        try:
            img = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"오류: 이미지 파일을 찾을 수 없습니다: {image_path}")
            return
        except Exception as e:
            print(f"오류: 이미지 로딩 중 오류 발생: {e}")
            return

        if transform is None:
            # 기본 검증용 변환 사용
            transform = transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

        image_tensor = transform(img).unsqueeze(0).to(device)
        model.eval()

        with torch.no_grad():
            output = model(image_tensor)
            probabilities = torch.nn.functional.softmax(output, dim=1)[0]
            score, pred_idx = torch.max(output, 1)
            predicted_class_idx = pred_idx.item()

        # 결과 시각화
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img); plt.title('입력 이미지'); plt.axis('off')

        plt.subplot(1, 2, 2)
        top_5_prob, top_5_idx = torch.topk(probabilities, 5)
        top_5_prob = top_5_prob.cpu().numpy()
        top_5_idx = top_5_idx.cpu().numpy()
        y_pos = np.arange(len(top_5_idx))
        plt.barh(y_pos, top_5_prob, align='center', color='lightcoral');
        plt.yticks(y_pos, [classes[i] for i in top_5_idx]); plt.xlabel('확률');
        plt.title('상위 5개 예측'); plt.gca().invert_yaxis();
        plt.tight_layout(); plt.show()

        predicted_class_name = classes[predicted_class_idx]
        confidence = probabilities[predicted_class_idx].item()
        print(f"
예측 결과: {predicted_class_name} (확신도: {confidence:.2%})")

    # 예측 함수 실행
    predict_custom_image(eval_model, custom_image_path, classes, device)

elif not custom_image_path:
    print("
커스텀 이미지 경로가 지정되지 않았습니다. 예측을 건너<0xEB>고 뜁니다.")
elif 'eval_model' not in locals():
     print("
평가 모델(eval_model)이 로드되지 않아 커스텀 이미지 예측을 건너<0xEB>고 뜁니다.")
elif not os.path.exists(custom_image_path):
      print(f"
오류: 지정된 이미지 경로 '{custom_image_path}'를 찾을 수 없습니다.")

## 20. 요약 및 향후 개선 방향

최종 학습 결과와 모델 성능을 요약하고, 추가적으로 시도해볼 수 있는 개선 방향을 제시합니다.

In [ ]:
# 학습 요약 및 향후 개선 방향
print("=" * 60)
print("종합 요약 및 향후 개선 방향")
print("=" * 60)

if 'config' in locals():
    print("[학습 설정]")
    for key, value in config.items():
        print(f"  - {key}: {value}")
else:
    print("[학습 설정] 정보 없음")

if 'history' in locals() and history['val_acc']:
    best_epoch_idx = np.argmax(history['val_acc'])
    best_val_acc = history['val_acc'][best_epoch_idx]
    print("
[최고 성능 에폭]")
    print(f"  - 에폭 번호: {best_epoch_idx + 1}")
    print(f"  - 검증 정확도: {best_val_acc:.2f}%")
else:
    print("
[최고 성능 에폭] 정보 없음")

if 'final_accuracy' in locals():
     print("
[최종 모델 성능]")
     print(f"  - 최종 검증 정확도: {final_accuracy:.2f}%")
     if 'class_accuracies' in locals():
         print(f"  - 클래스별 정확도 (평균): {np.mean(class_accuracies):.2f}%")
         print(f"  - 클래스별 정확도 (최저): {min(class_accuracies):.2f}%")
         print(f"  - 클래스별 정확도 (최고): {max(class_accuracies):.2f}%")
else:
     print("
[최종 모델 성능] 정보 없음")


print("
[향후 개선 방향 제안]")
suggestions = [
    "더 많은 에폭으로 학습 진행 또는 조기 종료(Early Stopping) 기준 조정",
    "데이터 증강(Data Augmentation) 기법 강화 또는 변경 (e.g., AutoAugment, CutMix)",
    "학습률 스케줄러(Learning Rate Scheduler) 변경 또는 파라미터 튜닝 (e.g., CosineAnnealingLR, OneCycleLR)",
    "옵티마이저(Optimizer) 변경 또는 파라미터 튜닝 (e.g., AdamW, SGD with Nesterov)",
    "모델 아키텍처 변경 또는 더 큰 사전 학습 모델 사용 (e.g., ResNet50, EfficientNet, ViT)",
    "하이퍼파라미터 자동 튜닝 도구 사용 (e.g., Weights & Biases Sweeps, Optuna)",
    "앙상블(Ensemble) 기법 적용: 여러 모델의 예측 결합",
    "정규화(Regularization) 기법 추가/조정 (e.g., Label Smoothing, Dropout 비율 변경, Weight Decay 값 조정)",
    "데이터셋 확장 또는 외부 데이터 활용",
    "모델 해석 기법(XAI)을 활용한 추가 분석 (e.g., SHAP)"
]
for i, suggestion in enumerate(suggestions):
    print(f" {i+1}. {suggestion}")

print("=" * 60)